In [1]:
import dill
import time
import random
import numpy as np
from sklearn.metrics import roc_curve, auc

import nltk

nltk.download("punkt")
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn

from torchtext.data import Field
from torchtext.data import TabularDataset
from torchtext.data import BucketIterator
from torchtext.data import Iterator

RANDOM_SEED = 2022
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

DATA_PATH = "./dataset"

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mary\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
TEXT = Field(
    sequential=True,
    use_vocab=True,
    tokenize=word_tokenize,
    lower=True,
    batch_first=True,
)
LABEL = Field(
    sequential=False,
    use_vocab=False,
    batch_first=True,
)

In [4]:
train_data, val_data, test_data = TabularDataset.splits(
    path=DATA_PATH,
    train="processed_train.tsv",
    validation="processed_val.tsv",
    test="test_no_label.tsv",
    format="tsv",
    fields=[("text", TEXT), ("label", LABEL)],
    skip_header=1,
)

train_iterator, val_iterator, test_iterator = BucketIterator.splits(
    (train_data, val_data, test_data),
    batch_size=8,
    device=None,
    sort=False,
)

TEXT.build_vocab(train_data, min_freq=2)

In [5]:
class LSTMClassifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, num_layers, pad_idx):
        super().__init__()
        self.embed_layer = nn.Embedding(
            num_embeddings=num_embeddings,
            embedding_dim=embedding_dim,
            padding_idx=pad_idx
        )
        self.lstm_layer = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=True,
            dropout=0.5
        )
        self.last_layer = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        embed_x = self.embed_layer(x)
        output, (_, _) = self.lstm_layer(embed_x)
        last_output = output[:, -1, :]
        last_output = self.last_layer(last_output)
        return last_output

In [11]:
def train(model: nn.Module, iterator: Iterator, optimizer: torch.optim.Optimizer, criterion: nn.Module, device: str):
    model.train()
    epoch_loss = 0

    for _, batch in enumerate(iterator):
        optimizer.zero_grad()

        text = batch.text
        if text.shape[0] > 1:
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)
            output = model(text).flatten()
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module, iterator: Iterator, criterion: nn.Module, device: str):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for _, batch in enumerate(iterator):
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)
            output = model(text).flatten()
            loss = criterion(output, label)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def test(model: nn.Module, iterator: Iterator, device: str):
    model.eval()
    with torch.no_grad():
        y_real = []
        y_pred = []
        for batch in iterator:
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            output = model(text).flatten().cpu()
            y_real += [label]
            y_pred += [output]
        y_real = torch.cat(y_real)
        y_pred = torch.cat(y_pred)
    
    fpr, tpr, _ = roc_curve(y_real, y_pred)
    auroc = auc(fpr, tpr)

    return auroc


def epoch_time(start_time: int, end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [10]:
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
N_EPOCHS = 20

lstm_classifier = LSTMClassifier(
    num_embeddings=len(TEXT.vocab),
    embedding_dim=100,
    hidden_size=200,
    num_layers=4,
    pad_idx=PAD_IDX,
)
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"
_ = lstm_classifier.to(device)

optimizer = torch.optim.Adam(lstm_classifier.parameters())
bce_loss_fn = nn.BCELoss()

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(lstm_classifier, train_iterator, optimizer, bce_loss_fn, device)
    valid_loss = evaluate(lstm_classifier, val_iterator, bce_loss_fn, device)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s")
    print(f"\tTrain Loss: {train_loss:.5f}")
    print(f"\t Val. Loss: {valid_loss:.5f}")

Epoch: 01 | Time: 0m 2s
	Train Loss: 0.52900
	 Val. Loss: 0.50881
Epoch: 02 | Time: 0m 2s
	Train Loss: 0.51905
	 Val. Loss: 0.48830
Epoch: 03 | Time: 0m 2s
	Train Loss: 0.48085
	 Val. Loss: 0.48175
Epoch: 04 | Time: 0m 2s
	Train Loss: 0.48974
	 Val. Loss: 0.48288
Epoch: 05 | Time: 0m 2s
	Train Loss: 0.49602
	 Val. Loss: 0.47809
Epoch: 06 | Time: 0m 2s
	Train Loss: 0.48480
	 Val. Loss: 0.48463
Epoch: 07 | Time: 0m 2s
	Train Loss: 0.49344
	 Val. Loss: 0.48789
Epoch: 08 | Time: 0m 2s
	Train Loss: 0.49612
	 Val. Loss: 0.48755
Epoch: 09 | Time: 0m 2s
	Train Loss: 0.48569
	 Val. Loss: 0.48816
Epoch: 10 | Time: 0m 2s
	Train Loss: 0.49290
	 Val. Loss: 0.49366
Epoch: 11 | Time: 0m 2s
	Train Loss: 0.49274
	 Val. Loss: 0.49266
Epoch: 12 | Time: 0m 2s
	Train Loss: 0.49266
	 Val. Loss: 0.48659
Epoch: 13 | Time: 0m 2s
	Train Loss: 0.47399
	 Val. Loss: 0.48968
Epoch: 14 | Time: 0m 2s
	Train Loss: 0.47757
	 Val. Loss: 0.49009
Epoch: 15 | Time: 0m 2s
	Train Loss: 0.49756
	 Val. Loss: 0.50383
Epoch: 16 

In [13]:
test_auroc = test(lstm_classifier, test_iterator, 'cpu')
print(f"testset score: {test_auroc:.5f}")

AttributeError: 'Example' object has no attribute 'label'